In [1]:
import os
import pandas as pd
import re
import json
from itertools import combinations

# cambiar url para leer los datos de otro subfolder con path relativo
# os.chdir('../')

# Importar stop words en espanol
from nltk.corpus import stopwords
spanish_stop_words = set(stopwords.words('spanish'))

In [ ]:
# Diccionario de siglas a eliminar dentro del nombre del cliente
# Se ejecuta solo una vez solo si se requiere actualizar la lista de siglas a elimnar

# lista_companias_filtro = ["CORP",'PLC','EPS','CI','SCA','INC','P.A.C','GROUP','SA','SAS','LTDA','LTD','LIMITED','ESP','SOCIEDAD','LP','BV','BIC']
# diccionario = {}
# diccionario["siglas"] = [word.lower() for word in lista_companias_filtro]

# with open("data/archivos_auxiliares/diccionario_siglas.json","w") as f:
#     json.dump(diccionario, f)

In [2]:
# importacion de datos
noticias = pd.read_csv('../datos/noticias.csv')
clientes = pd.read_csv('../datos/clientes.csv')
clientes_noticias = pd.read_csv('../datos/clientes_noticias.csv')
with open("data/archivos_auxiliares/diccionario_siglas.json", 'r') as f:
    diccionario_siglas = json.load(f)
    lista_siglas = diccionario_siglas['siglas']

In [3]:
# funciones utiles
from distutils.command.clean import clean

def check_cliente(cliente, noticia, sector):

    """
        Valida si dentro de un string aparece un texto en concreto.
        Mas espcificamente, esta funcion analiza si dentro de una noticia aparece un cliente.

        Inputs:

            cliente: str

                String con el nombre del cliente.
            
            noticica: str

                String con la noticia

        Outputs: str

            Categorias dependindo de los siguientes criterios

            Si el nombre del cliente es compuesto y tiene menos de 2 palabras, se analiza si el nombre de ese cliente aparece dentro de la noticia o no, buscando directamente ocurrencias simultaneas de ambas palabras. ejemplo: cliente = 'banco republica". Si el cliente es mencionado en la noticia se crea la categoria 'Cliente', de lo contrario 'No Aplica'

            En caso en el que el nombre del cliente tenga mas de 2 palabras ocurren varios casos.

                1. Se generan combinaciones de dos palabras del nombre del cliente y se buscan dentro del texto. Si esa combinacion aparece dentro de la noticia se crea la categoria 'Cliente_'
        
    """

    cliente_edit = cliente.split(" ")

    if len(cliente_edit)<3:

        condicion = bool(re.findall(cliente,noticia))
        if condicion:
            output = 'Cliente'
        else:
            output = 'No Aplica'

    else:
        combs = [ " ".join(list(x)) for x in combinations(cliente_edit, 2)]
        condicion = [bool(re.findall(x,noticia)) for x in combs]

        if True in condicion:
            output = 'Cliente'
            
        else:
            output = "No Aplica"

    if (sector>0.5) and (output == 'No Aplica'):
    # toca revisar como construirlo de manera correcta, el sector debe ir despues de asiganar a todos los clientes
        output = 'Sector'   

    return output

def clean_name(text,lista_filtro):

    """
    Esta funcion limpia una cadena de texto excluendo todas aquellas palabras que aparecen dentro de lista_filtro.
    Se usa por defecto la separación de la cadena con espacios en blanco.

    Inputs: 

        text : str

            Texto a limpiar
        
        lista_filtro : list

            Lista de palabras a remover
        
    Outputs:

        String con la cadena de texto limpia

    """
    lista_nombre = text.split(' ')
    nombre_final = [word for word in lista_nombre if word not in lista_filtro and word not in spanish_stop_words]
    return ' '.join(nombre_final)

def elimina_letras_sueltas(text):
    """
        Elimina letras unicas dentro de un texto, por defecto el separador de la cadena de texto son los espacios en blanco.

        Inputs:

            text: str

                String al que se eliminaran las letras suletas

        Outputs: str

            String con el filtrado de caracteres alphabeticos sueltos (De longitud igual a uno)

    """
    text_clean = text.split(" ")
    text_clean = [word for word in text_clean if len(word)>1]

    return " ".join(text_clean)


def clean_df(df, *args, **kwards):

    """
        Esta funcion es auxiliar y permite aplicar funciones previas creadas directamente sobre un dataframe para depurar especificamente campos como el nombre del cliente y generar otro tipo de feature sobre ese nombre.

        Inputs:

            df: pandas.DataFrame
                DataFrame al que se le aplicaran los cambios

            *args and **kwards:
                Argumentos de funciones propias usadas dentro del mismo
        
        Outputs: pandas.DataFrame

            DataFrame de pandas con las correcciones realizadas
    """

    lista_columnas = ['desc_ciiu_division','desc_ciuu_grupo','desc_ciiuu_clase','subsec']

    df_clean = df.copy()
    df_clean['nombre_clean'] = df_clean['nombre'].apply(lambda x: x.replace('.',''))
    df_clean['nombre_clean'] = df_clean['nombre_clean'].apply(lambda x: clean_name(text = x.lower(),*args, **kwards))
    # df_clean['nombre_clean'] = df_clean['nombre_clean'].apply(elimina_letras_sueltas)
    df_clean['nombre_clean'] = df_clean['nombre_clean'].str.title()
    df_clean['long_nombre'] = df_clean['nombre_clean'].apply(lambda x: len(x))
    df_clean['palabras_nombre'] = df_clean['nombre_clean'].apply(lambda x: len(x.split(' ')))


    df_clean['lista_ciiud'] = df_clean[lista_columnas].agg(" ".join, axis = 1)
    df_clean['lista_ciiud'] = df_clean['lista_ciiud'].apply(lambda x: clean_name(x.lower(),[]))
    df_clean['lista_ciiud'] = df_clean['lista_ciiud'].apply(clean_puntuation)

    df_clean = df_clean[['nit',"nombre_clean",'lista_ciiud']]

    return df_clean

def preprocessing_noticas_clientes(x):

    x['Participacion'] = x.apply(lambda x: check_cliente(cliente=x['nombre_clean'], noticia=x['news_text_content']))    
    return x

def noticia_cliente_detalle(noticias,clientes_noticias):

    """

        Esta funcion realiza un merge sobre 2 pandas.DataFrame, posteriormente filtra ciertos campos necesarios. Funcion auxiliar. 

        Inputs:

            noticias: pandas.DataFrame

                Primer df para hacer el merge
            
            clientes: pandas.DataFrame

                Segundo df para hacer el merge
        
        Outputs: pandas.DataFrame

            DataFrame con las variables preseleccioandas sobre el merge de clientes y noticias
        
    """

    final_df = clientes_noticias.merge(noticias, on = 'news_id', how='left')
    final_df = final_df[['nit','news_id', 'news_text_content']]
    return final_df

def clean_puntuation(text):

    text_ = re.sub(r'[^\w\s]', '', text)
    text_ = text_.split(' ')
    text_ = list(set(text_))

    return text_

def conteo_palbras_lista(text, noticia):
    contador = [bool(re.findall(word,noticia)) for word in text]
    return contador.count(True)

In [4]:
# Limpieza de dfs
clientes_clean = clean_df(clientes, lista_filtro = lista_siglas)
noticia_cliente_detalle_df = noticia_cliente_detalle(noticias,clientes_noticias)
participacion_df = clientes_clean.merge(noticia_cliente_detalle_df, on = 'nit', how = 'left')

# Quitar stopr words de noticas para buscar nombre en mayusc de la empresa
participacion_df['cuerpo_not_clean'] = participacion_df['news_text_content'].apply(lambda x: clean_name(text = x, lista_filtro=[]))

participacion_df["Sector_indice"] = participacion_df.apply(lambda x: conteo_palbras_lista(text=x['lista_ciiud'], noticia=x['cuerpo_not_clean'].lower()), axis=1)    
participacion_df["Conteo_Ciiu"] = participacion_df['lista_ciiud'].apply(len)
participacion_df["prop_sector"] = round(participacion_df["Sector_indice"]/participacion_df["Conteo_Ciiu"],2)
participacion_df["Participacion"] = participacion_df.apply(lambda x: check_cliente(cliente=x['nombre_clean'], noticia=x['cuerpo_not_clean'], sector = x['prop_sector']), axis=1)    

In [5]:
participacion_df

,nit,nombre_clean,lista_ciiud,news_id,news_text_content,cuerpo_not_clean,Sector_indice,Conteo_Ciiu,prop_sector,Participacion
0,805027024,Supermercado Gran Colombia,"[conservacion, alimenticios, pescado, producto...",news12584,"En marzo de 2020, la pandemia encontro a Marco...","En marzo 2020, pandemia encontro Marcos Acuna ...",3,12,0.25,No Aplica
1,805027024,Supermercado Gran Colombia,"[conservacion, alimenticios, pescado, producto...",news12758,Olimpica S.A. informo que en alianza con Plan ...,Olimpica S.A. informo alianza Plan B Investmen...,1,12,0.08,No Aplica
2,805027024,Supermercado Gran Colombia,"[conservacion, alimenticios, pescado, producto...",news13241,La cadena de supermercados estadounidense The ...,La cadena supermercados estadounidense The Fre...,3,12,0.25,No Aplica
3,805027024,Supermercado Gran Colombia,"[conservacion, alimenticios, pescado, producto...",news15048,"'Riqueza Natural', el primer programa de gran ...","'Riqueza Natural', primer programa gran escala...",3,12,0.25,No Aplica
4,805027024,Supermercado Gran Colombia,"[conservacion, alimenticios, pescado, producto...",news15611,"El 2 de julio ultimo, antes de que Cristina Fe...","El 2 julio ultimo, Cristina Fernandez Kirchner...",3,12,0.25,No Aplica
...,...,...,...,...,...,...,...,...,...,...
74612,891102723,Mecanicos Asociados,"[explotacion, canteras, minas, gas, petroleo, ...",news99569,Este jueves en el Hall Central del Pabellon Ar...,Este jueves Hall Central Pabellon Argentina re...,0,10,0.00,No Aplica
74613,891102723,Mecanicos Asociados,"[explotacion, canteras, minas, gas, petroleo, ...",news99573,Fueron diez los proyectos seleccionados en las...,Fueron diez proyectos seleccionados lineas fin...,1,10,0.10,No Aplica
74614,891102723,Mecanicos Asociados,"[explotacion, canteras, minas, gas, petroleo, ...",news99574,Tras dos anos de cierre por la emergencia sani...,Tras dos anos cierre emergencia sanitaria COVI...,0,10,0.00,No Aplica
74615,891102723,Mecanicos Asociados,"[explotacion, canteras, minas, gas, petroleo, ...",news99576,"Sabado, Septiembre 17 de 2022 La economia nara...","Sabado, Septiembre 17 2022 La economia naranja...",3,10,0.30,No Aplica


In [6]:
participacion_df['Participacion'].value_counts()

No Aplica    64853
Sector        5551
Cliente       4213
Name: Participacion, dtype: int64

In [7]:
a = participacion_df[participacion_df['Participacion']!='No Aplica'].groupby('nit').count()['prop_sector'].reset_index()
print(a.shape[0]/clientes.shape[0])
a.sort_values('prop_sector',ascending = False).head(20)

0.7869940278699403


,nit,prop_sector
43,800091063,92
399,860007386,79
822,899999032,71
451,860028971,69
414,860012357,65
826,899999063,62
601,890201213,61
823,899999034,60
827,899999068,58
586,890104633,57


In [179]:
participacion_df[participacion_df['nit']==860007386]
# participacion_df[(participacion_df['nit']==800091063) & (participacion_df['Participacion']=='Sector')]

,nit,nombre_clean,lista_ciiud,news_id,news_text_content,cuerpo_not_clean,Sector_indice,Conteo_Ciiu,prop_sector,Participacion
63778,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news13192,La revista Times Higher Education (THE) dio a ...,La revista Times Higher Education (THE) dio co...,3,5,0.6,Cliente
63779,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news13193,Diez universidades de Colombia estan dentro de...,Diez universidades Colombia estan dentro 100 m...,3,5,0.6,Sector
63780,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news15316,"--La economia naranja es un fracaso--, dice ho...","--La economia naranja fracaso--, dice hoy Davi...",2,5,0.4,No Aplica
63781,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news15607,El presidente Gustavo Petro ya tiene listo a c...,El presidente Gustavo Petro listo casi gabinet...,1,5,0.2,Cliente
63782,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news15643,El nuevo presidente Gustavo Petro posesiono a ...,El nuevo presidente Gustavo Petro posesiono mi...,2,5,0.4,Cliente
...,...,...,...,...,...,...,...,...,...,...
63925,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news99385,Con presencia del ministro de Educacion se rea...,Con presencia ministro Educacion realizo jorna...,2,5,0.4,No Aplica
63926,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news99386,"El miercoles 10 de agosto, el expresidente de ...","El miercoles 10 agosto, expresidente Colombia ...",1,5,0.2,No Aplica
63927,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news99387,?Deberia o no ensenarse sobre el conflicto arm...,?Deberia ensenarse conflicto armado colombiano...,2,5,0.4,No Aplica
63928,860007386,Universidad Andes,"[superior, educacion, educativos, universidade...",news99389,"El flamante presidente de Colombia, Gustavo Pe...","El flamante presidente Colombia, Gustavo Petro...",1,5,0.2,No Aplica


In [158]:
noticias[noticias['news_id']=='news13193'].reset_index().loc[0,"news_url_absolute"]

'https://www.infobae.com/america/colombia/2022/07/19/nueve-universidades-colombianas-estan-entre-las-100-mejores-de-america-latina/'

In [159]:
# La siguientes lineas fueron pruebas anteriores y estan en revision
# se recomienda no ejecutarlas
clientes['nombre_editado'] = clientes['nombre'].str.lower()
clientes['val_nombre_editado'] = clientes['nombre_editado'].apply(lambda x: bool(re.findall('davivienda',x)))
clientes[clientes['val_nombre_editado']]
noticias['nombre_editado'] = noticias['news_text_content'].str.lower()
noticias['val_nombre_editado'] = noticias['nombre_editado'].apply(lambda x: bool(re.findall('eps',x)))
davi = noticias[noticias['val_nombre_editado']].copy()
davi = davi.merge(clientes_noticias, on = 'news_id', how = 'left')
davi.loc[1,'news_text_content']
clientes[clientes['nit'].isin(list(davi['nit'].unique()))]
x = davi[davi['nit']==805027024].copy().reset_index()
x.loc[0,'news_url_absolute_x'] # news_url_absolute_x, news_title, news_text_content	